In [1]:
from platform import python_version
print(f'Jupyter Notebook on Mac OS\nPython Version: {python_version()}')

Jupyter Notebook on Mac OS
Python Version: 3.9.12


# D212 - Association Rules & Lift Analysis

In [2]:
## Import libraries/packages
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, warnings
sns.set_palette('colorblind')
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,10)
plt.rcParams['figure.max_open_warning'] = False
warnings.filterwarnings('ignore')

from mlxtend.frequent_patterns import apriori, association_rules

## Data Preparation
---

In [3]:
## Import data
df = pd.read_csv('teleco_market_basket.csv')

In [4]:
## View shape of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [5]:
## Review data table
df.head()

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## Make wide format df into list of transactions
transactions = []
for _, row in df.iterrows():
    transaction = row.dropna().tolist()
    transactions.append(transaction)

In [7]:
## Find empty transactions
n = 0
for i in range(len(transactions)):
    if transactions[i] == []:
        n+=1
print(f'Empty Transactions: {n}')

Empty Transactions: 7501


In [8]:
## Remove empty transactions
for i in range(n):
    transactions.remove([])

In [9]:
## Insert filtered transactions into a dataframe
df = pd.DataFrame(transactions, columns=df.columns)

In [10]:
## Review dataframe
df.head()

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
1,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,UNEN Mfi Certified 5-pack Lightning Cable,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Cat8 Ethernet Cable,HP 65 ink,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Dust-Off Compressed Gas 2 pack,Screen Mom Screen Cleaner kit,Moread HDMI to VGA Adapter,HP 62XL Tri-Color ink,Apple USB-C Charger cable,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [11]:
## Create list of unique items
items = set()
for col in df.columns:
    for i in range(len(df)):
        if df[col][i] != None:
            items.add(df[col][i])

In [12]:
## Create binary list of transactions
transactions = []
for _, row in df.iterrows():
    transaction = []
    for item in items:
        if item in row.values:
            transaction.append(True)
        else:
            transaction.append(False)
    transactions.append(transaction)

In [13]:
## Create dataframe with binary list
df = pd.DataFrame(transactions, columns=items)

In [14]:
## Review clean df
df

,Cat8 Ethernet Cable,seenda Wireless mouse,Premium Nylon USB Cable,Stylus Pen for iPad,DisplayPort ot HDMI adapter,SAMSUNG EVO 32GB card,SanDisk 32GB card,HooToo USB C Hub,Syntech USB C to USB Adapter,iPhone 12 case,...,SanDisk 128GB Ultra microSDXC card,CicTsing MM057 2.4G Wireless Mouse,Dust-Off Compressed Gas,Blue Light Blocking Glasses,HP 61 ink,Moread HDMI to VGA Adapter,SAMSUNG EVO 64GB card,HP 65 ink,10ft iPHone Charger Cable 2 Pack,SanDisk Extreme 32GB 2pack card
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
## Store clean data
df.to_csv('clean_transactions.csv')

In [16]:
## Gather one example of transactions in the dataset
trans_num = [4, 5, 6, 7]
transaction = []
for num in trans_num:
    items = []
    for i, item in enumerate(transactions[num]):
        if item == True:
            items.append(df.columns[i])
    transaction.append(items)
trans_df = pd.DataFrame(
    transaction, 
    columns=[f'Item {n}' for n in range(1, len(transaction[0])+1)], 
    index=[f'Transaction {n}' for n in trans_num]
).fillna('').transpose()
trans_df

,Transaction 4,Transaction 5,Transaction 6,Transaction 7
Item 1,Screen Mom Screen Cleaner kit,TopMate C5 Laptop Cooler pad,USB 2.0 Printer cable,HP952XL ink
Item 2,HP 62XL Tri-Color ink,,Anker 2-in-1 USB Card Reader,Apple Lightning to USB cable
Item 3,Apple USB-C Charger cable,,,10ft iPHone Charger Cable 2 Pack
Item 4,Dust-Off Compressed Gas 2 pack,,,
Item 5,Moread HDMI to VGA Adapter,,,


## Data Analysis
---

In [17]:
## Use apriori to get list of most frequently paired items
frq_items = apriori(df, min_support = 0.05, use_colnames = True)

In [18]:
## Review list
frq_items

,support,itemsets
0,0.062525,(Cat8 Ethernet Cable)
1,0.051060,(Premium Nylon USB Cable)
2,0.095054,(Stylus Pen for iPad)
3,0.052393,(SAMSUNG EVO 32GB card)
4,0.081056,(Syntech USB C to USB Adapter)
5,0.068391,(Anker USB C to HDMI Adapter)
6,0.129583,(Screen Mom Screen Cleaner kit)
7,0.170911,(USB 2.0 Printer cable)
8,0.076523,(TopMate C5 Laptop Cooler pad)
9,0.087188,(Apple Lightning to Digital AV Adapter)


In [19]:
## Collect rules and sort by confidence, lift
rules = association_rules(frq_items, metric='lift')
rules = rules.sort_values(['lift', 'support', 'confidence'], ascending =[False, False, False]).reset_index(drop=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(VIVO Dual LCD Monitor Desk mount),(Dust-Off Compressed Gas 2 pack),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(Dust-Off Compressed Gas 2 pack),(VIVO Dual LCD Monitor Desk mount),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(HP 61 ink),(Dust-Off Compressed Gas 2 pack),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(Dust-Off Compressed Gas 2 pack),(HP 61 ink),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(Apple Pencil),(Dust-Off Compressed Gas 2 pack),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
5,(Dust-Off Compressed Gas 2 pack),(Apple Pencil),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158
